# LangGraph Section 5 Notes - Advanced RAG Flows

* Be extremely careful with version dependencies, as LangGraph and LangChain are in active development, and very sensitive to versions

# Step 1 - Ingestion Process

In [10]:
# Solution will use vector store and ChromaDB (since it's local, fast and non persistent)

In [11]:
# Import statements:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings

In [12]:
load_dotenv()

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Creates vector store and splits the documents into it
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
    persist_directory="./.chroma",
)

# This will be our retriever, but we will have persistence here via /chroma folder
retriever = Chroma(
    collection_name="rag-chroma",
    persist_directory="./.chroma",
    embedding_function=OpenAIEmbeddings(),
).as_retriever()

# Step 2 - Definition of State

In [18]:
from typing import List, TypedDict

# States in LangGraph (aka what is rememebred across the graph) - is defined as a TypedDict object
# TypeDict will set

# Documents will be a list of strings; all other properties is a string

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: bool
    documents: List[str]

# Side Note - Invocation Operation

* Invoke has different meanings in LangGraph and LangChain.
* Depending on the context and object, invoke has different properties and usage:

In [ ]:
# # Retriever's invoke: "find relevant documents"
# docs = retriever.invoke("query")

# # LLM's invoke: "generate text"
# response = llm.invoke("What is 2+2?")

# # Embeddings' invoke: "convert text to vectors"
# vectors = embeddings.invoke("convert this to numbers")

# # Prompt template's invoke: "fill in the template"
# filled = prompt.invoke({"question": "What is life?", "context": "..."})

# # Parser's invoke: "parse the output"
# parsed = output_parser.invoke("Raw text to parse")

# Runnable PassThrough

In [ ]:
# Telling the pipeline in LG and LC to pass something or keep it th

In [32]:
# from langchain.schema import RunnablePassthrough
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate

# # WITHOUT RunnablePassthrough ❌
# basic_chain = (
#     retriever  
#     | ChatPromptTemplate.from_template("Context: {context}\nPlease summarize it.")
#     | ChatOpenAI()
# )
# # Flow:
# # 1. User asks: "Who was Einstein?"
# # 2. Retriever gets documents → [docs about Einstein]
# # 3. Template only gets {context} → "Context: [docs about Einstein]"
# # 4. LLM only sees the context, original question is lost!


# # WITH RunnablePassthrough ✅
# better_chain = (
#     {
#         "context": retriever,          # Gets relevant docs
#         "question": RunnablePassthrough()  # Preserves "Who was Einstein?"
#     }
#     | ChatPromptTemplate.from_template("""
#         Context: {context}
#         Question: {question}    # Can still use original question here!
#         Please answer the question.
#         """)
#     | ChatOpenAI()
# )
# # Flow:
# # 1. User asks: "Who was Einstein?"
# # 2. Retriever gets documents → {"context": [docs]}
# # 3. RunnablePassthrough keeps question → {"question": "Who was Einstein?"}
# # 4. Template gets both → "Context: [docs], Question: Who was Einstein?"
# # 5. LLM sees both context AND original question!

In [ ]:
# Best practices for piping:

# Follow this structure:
# Simple Chain
loader | splitter | embedder

# RAG Chain
retriever | template | llm

# RAG with preserved data
{
    "context": retriever,
    "question": RunnablePassthrough()
} | template | llm

# Multi-source RAG
{
    "web": web_retriever,
    "docs": doc_retriever,
    "question": RunnablePassthrough()
} | template | llm

# Step 3 - Retriever Node

* Technically vector databases will handle this for you, but let's abstract this lower and create a node that does it for you, for learning prposes!

In [20]:
# Retriever will refer to the vector store
from typing import Any, Dict

# From LangGraph family
from graph.state import GraphState

# let's call the ingestion file we made and use the retriver code
from ingestion import retriever # import from local Python file, retriever object

In [ ]:
def retrieve(GraphState) -> Dict[str, Any]: # retrieve node, takes in Graph State. Output is dir with str or any
    print("---RETRIEVING FOR YOU!---")
    question = state["question"] # one of the defined attributes
    
    # Use the retriever here:
    documents = retriever.invoke(question)